# Send Data from Bronze Table to Silver VAUsage Table
To run this notebook, import it into Azure Synapse and attach it to an Apache Spark Pool.      
Choose the "Small" Node Size, and choose "3" as the Number of Nodes.     
Be sure to run the "rate-streaming-to-bronze" Notebook beforehand, to ensure there is data to pull from.

In [ ]:
%%spark
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

## Configure the Storage Account (to read from)
Replace the value `<storageAccountName>` with the name of the storage account where the Bronze Delta Table data is stored.   

In [ ]:
%%spark
val storageAccountName = "<storageAccountName>"
val bronzeDataLocation: String = "abfss://datalake@"+storageAccountName+".dfs.core.windows.net/bronzeSynapse"  
val silverDataLocation: String = "abfss://datalake@"+storageAccountName+".dfs.core.windows.net/silverSynapse/VAUsage"  

## Read the Data
Here the data is read from the `bronzeDataLocation` specified in the previous cell, which is configured using the value inputted for `storageAccount`.

In [ ]:
%%spark
var bronzeDF = spark.readStream.format("delta").load(bronzeDataLocation)

## Parse the Body and Split into Columns
The schema of the Dataframe is configured to match the schema of the Silver VAUsage Table. The body is parsed and split into columns.



In [ ]:
%%spark
val silverSchema: StructType = new StructType().add("Object", StringType).add("Action", StringType)

var silverDF = bronzeDF.where("Properties.topic == 'VAUsage'")

silverDF = silverDF.withColumn("Body", col("Body").cast(StringType)) // cast the "body" column to StringType
silverDF = silverDF.withColumn("JsonBody", get_json_object(col("Body"), "$")) // extracts JSON object from the "body" column
silverDF = silverDF.withColumn("SilverSchemaFields", from_json(col("JsonBody"), silverSchema)) // returns a struct value with the given JSON string and schema

silverDF = silverDF.select(
    col("ProcessedTimestamp"),
    col("ProcessedDate"),
    col("ProcessedHour"),
    col("UserId"),
    col("Properties"),     
    col("SilverSchemaFields.*") // creates a new column for each struct field from 'silverSchema'     
)      

silverDF.printSchema()

## Write Data to Silver VAUsage Table

In [ ]:
%%spark
val silverVAUsageQuery = silverDF.writeStream.format("delta").
outputMode("append").
partitionBy("ProcessedDate", "ProcessedHour").
option("checkpointLocation", silverDataLocation + "/checkpoint").
start(silverDataLocation)

## Viewing the Data

In [ ]:
%%spark
var silverViewDF = spark.read.format("delta").load(silverDataLocation)
silverViewDF.orderBy(col("ProcessedTimestamp").desc).show()

In [ ]:
%%spark
silverViewDF.printSchema()

In [ ]:
%%spark
display(silverViewDF.orderBy(col("ProcessedTimestamp").desc))